In [2]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화해."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 저녁 같이 먹을까요?"),
]

model.invoke(messages)

AIMessage(content='안녕, 개스톤. 너의 초대는 고맙지만, 오늘 저녁은 사양할게. 내가 지금 읽고 있는 책이 너무 재미있어서, 그 이야기에 푹 빠져있거든. 우리가 공통으로 좋아하는 취미나 이야기를 나누는 것도 좋을 것 같아. 혹시 어떤 책을 좋아하니?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 55, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4j5Yi1sYfeUGK9bIcdNOxqRqWDLx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8478b984-d125-4a7a-b7ae-7f95a2b6c34f-0', usage_metadata={'input_tokens': 55, 'output_tokens': 80, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'안녕하세요, 개스톤. 당신의 제안은 감사하지만, 오늘 저녁엔 다른 계획이 있어요. 책을 읽는 것이나 아버지를 돕는 일이 훨씬 더 흥미롭답니다. 하지만 좋은 하루 보내세요!📚'

#### LCEL (LangChain Expression Language) 이용

In [7]:
chain = model | parser
chain.invoke(messages)

'안녕하세요, 개스톤. 초대해줘서 고마워요. 하지만 저녁 약속은 이미 잡혀있어요. 매일 새로운 책을 읽느라 바쁘기도 하고요. 다음에 기회가 된다면 함께 이야기를 나누는 것도 좋을 것 같아요. 당신의 하루는 어땠나요?'

#### 프롬프트 템플릿 이용하기

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


In [9]:
chain = prompt_template | model | parser
chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'안녕하세요, 야수님. 네, 저녁 함께하면 좋을 것 같아요. 무엇을 준비하셨나요? 어디서 먹을지 생각해 놓으셨나요?'